In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [5]:
pair = "EUR_USD"
granularity = "H1"

In [6]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [8]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
#mod_cols

In [9]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [10]:
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,4625.081250,1.073498,1.074300,1.072716,1.073517,1.073405,1.074217,1.072622,1.073433,1.073591,1.074392,1.072800,1.073601
std,3388.282468,0.019954,0.019821,0.020082,0.019930,0.019955,0.019823,0.020083,0.019931,0.019953,0.019819,0.020081,0.019928
min,25.000000,0.994500,0.995160,0.993540,0.994480,0.994430,0.995090,0.993460,0.994410,0.994560,0.995250,0.993630,0.994540
25%,2370.500000,1.062050,1.062880,1.061117,1.062060,1.061978,1.062810,1.061030,1.061997,1.062130,1.062968,1.061190,1.062137
50%,3794.000000,1.074305,1.075080,1.073600,1.074350,1.074235,1.075005,1.073510,1.074280,1.074370,1.075220,1.073675,1.074430
75%,5935.500000,1.089417,1.090172,1.088792,1.089440,1.089338,1.090100,1.088703,1.089352,1.089495,1.090253,1.088870,1.089513
max,32171.000000,1.108500,1.109560,1.108050,1.108520,1.108420,1.109500,1.107970,1.108450,1.108570,1.109640,1.108130,1.108590


In [26]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-11-09T05:00:00.000000000Z,4138,1.00694,1.00732,1.00634,1.00684,1.00687,1.00725,1.00626,1.00676,1.00700,1.00741,1.00642,1.00693
1,2022-11-09T06:00:00.000000000Z,3939,1.00686,1.00706,1.00562,1.00585,1.00678,1.00698,1.00555,1.00577,1.00694,1.00713,1.00570,1.00593
2,2022-11-09T07:00:00.000000000Z,5669,1.00586,1.00716,1.00586,1.00644,1.00579,1.00708,1.00578,1.00636,1.00594,1.00724,1.00594,1.00652
3,2022-11-09T08:00:00.000000000Z,8802,1.00646,1.00854,1.00632,1.00694,1.00638,1.00846,1.00624,1.00686,1.00654,1.00862,1.00639,1.00702
4,2022-11-09T09:00:00.000000000Z,8525,1.00696,1.00709,1.00402,1.00470,1.00688,1.00702,1.00395,1.00462,1.00704,1.00716,1.00410,1.00477


In [35]:

df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
df_ma['MA_8'] = df_ma.mid_c.rolling(window=8).mean()
df_ma.dropna(inplace=True)
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8
7,2022-11-09T12:00:00.000000000Z,1.00542,1.00568,1.00406,1.00498,1.005716
8,2022-11-09T13:00:00.000000000Z,1.00494,1.00506,1.00228,1.00286,1.005219
9,2022-11-09T14:00:00.000000000Z,1.00294,1.00410,1.00110,1.00184,1.004717
10,2022-11-09T15:00:00.000000000Z,1.00184,1.00866,1.00125,1.00280,1.004263
11,2022-11-09T16:00:00.000000000Z,1.00278,1.00528,1.00278,1.00368,1.003855


In [36]:
df_plot = df_ma.iloc[-100:].copy()
df_plot.shape

(100, 6)

In [37]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot.MA_8,
    line=dict(color="#027FC3", width=2),
    line_shape='spline',
    name='MA_8'
    ))
fig.update_layout(width=1000,height=300,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

In [30]:
ma_list = [8,16,32,64,128,256]
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

df_ma.dropna(inplace=True)
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2022-11-23T20:00:00.000000000Z,1.04010,1.04034,1.03914,1.04026,1.037292,1.034874,1.032350,1.028816,1.032362,1.028219
256,2022-11-23T21:00:00.000000000Z,1.04029,1.04034,1.03934,1.03954,1.038085,1.035298,1.032772,1.028993,1.032365,1.028347
257,2022-11-23T22:00:00.000000000Z,1.03992,1.04012,1.03932,1.03958,1.038425,1.035701,1.033179,1.029206,1.032371,1.028478
258,2022-11-23T23:00:00.000000000Z,1.03971,1.04193,1.03971,1.04180,1.038945,1.036168,1.033600,1.029460,1.032382,1.028617
259,2022-11-24T00:00:00.000000000Z,1.04182,1.04192,1.04052,1.04152,1.039590,1.036802,1.034038,1.029711,1.032398,1.028752


In [31]:
df_plot = df_ma.iloc[-200:].copy()
df_plot.shape

(200, 11)

In [33]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=300,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

In [68]:
ma_list = [16,64]
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2022-11-11T20:00:00.000000000Z,1.03626,1.03626,1.03483,1.03575,1.029723,1.012882
1,2022-11-11T21:00:00.000000000Z,1.03576,1.03599,1.03526,1.03542,1.030567,1.013328
2,2022-11-13T22:00:00.000000000Z,1.03351,1.03401,1.03128,1.03194,1.031203,1.013736
3,2022-11-13T23:00:00.000000000Z,1.03205,1.03346,1.03146,1.03270,1.031718,1.014146
4,2022-11-14T00:00:00.000000000Z,1.03268,1.03366,1.03240,1.03328,1.032107,1.014558


In [69]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [70]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [71]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [72]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
89,2022-11-17T13:00:00.000000000Z,1.03326,1.03537,1.03108,1.03124,1.036911,1.037264,-0.000353,0.000137,True
172,2022-11-23T00:00:00.000000000Z,1.03109,1.03214,1.03040,1.03055,1.028721,1.028563,0.000158,-0.000135,True
243,2022-11-27T23:00:00.000000000Z,1.03775,1.03814,1.03656,1.03672,1.039567,1.039761,-0.000193,0.000107,True
261,2022-11-28T17:00:00.000000000Z,1.03852,1.03886,1.03685,1.03698,1.040813,1.040747,0.000066,-0.000152,True
268,2022-11-29T00:00:00.000000000Z,1.03436,1.03506,1.03395,1.03404,1.039486,1.039794,-0.000308,0.000206,True


In [73]:
df_plot = df_ma.iloc[(89-12):(89+12)].copy()
df_plot.shape

(24, 10)

In [74]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

In [3]:
df_all = pd.read_pickle("all_trades.pkl")

In [4]:
df_all.head()

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION
19,2022-11-10 00:00:00+00:00,1.13768,0.000449,-0.000466,1,10.0,10.0,GBP_USD,4,8,8.0
27,2022-11-10 08:00:00+00:00,1.13868,-0.000163,0.000385,-1,221.2,-221.2,GBP_USD,4,8,5.0
32,2022-11-10 13:00:00+00:00,1.16080,0.003005,-0.000534,1,42.7,42.7,GBP_USD,4,8,12.0
44,2022-11-11 01:00:00+00:00,1.16507,-0.000815,0.000771,-1,58.1,-58.1,GBP_USD,4,8,4.0
48,2022-11-11 05:00:00+00:00,1.17088,0.000375,-0.001102,1,88.0,88.0,GBP_USD,4,8,66.0


In [5]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,66616.000000,66616.000000,6.661600e+04,66616.000000,66616.000000,66616.000000,66616.000000,66616.000000,66616.000000
mean,36.659633,-0.000349,3.983899e-04,0.000120,3.333538,-6.447781,12.062748,62.616909,49.509502
std,59.320624,0.030876,2.967318e-02,1.000007,75.398324,75.196047,12.753577,64.973012,76.341077
min,0.544460,-0.826281,-6.562969e-01,-1.000000,-751.900000,-522.400000,4.000000,8.000000,1.000000
25%,0.901260,-0.000228,-2.225000e-04,-1.000000,-24.500000,-36.000000,4.000000,16.000000,6.000000
50%,1.343015,0.000000,-2.220446e-16,1.000000,1.600000,-12.400000,8.000000,32.000000,18.000000
75%,84.366000,0.000226,2.268750e-04,1.000000,29.700000,9.100000,16.000000,96.000000,66.000000
max,183.583000,0.622125,8.209063e-01,1.000000,852.200000,852.200000,64.000000,256.000000,1346.000000
